## Fetch Rewards Coding Exercise - Part Two and Three

### Using sqlite and python for examining the data for quality and writing sql queries for answering the below mentioned business questions:
#### When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?
#### When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

In [59]:
# Import Essential Libraries

import sqlite3
import json
import pandas as pd
from sqlite3 import Error
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Load JSON data into Pandas DataFrames

with open('users.json') as users:
    users_data = json.load(users)
    
with open('brands.json') as brands:
    brands_data = json.load(brands)
    
with open('receipts.json') as receipts:
    receipts_data = json.load(receipts)
    
users_df = pd.DataFrame(pd.json_normalize(users_data, sep = '_'))
brands_df = pd.DataFrame(pd.json_normalize(brands_data, sep = '_'))
receipts_df = pd.DataFrame(pd.json_normalize(receipts_data, sep = '_'))

### Perform Basic EDA to understand the datasets

In [3]:
brands_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   barcode       400 non-null    object
 1   category      346 non-null    object
 2   categoryCode  171 non-null    object
 3   name          400 non-null    object
 4   topBrand      200 non-null    object
 5   _id_$oid      400 non-null    object
 6   cpg_$id_$oid  400 non-null    object
 7   cpg_$ref      400 non-null    object
 8   brandCode     317 non-null    object
dtypes: object(9)
memory usage: 28.2+ KB


In [4]:
users_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495 entries, 0 to 494
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   active             495 non-null    bool   
 1   role               495 non-null    object 
 2   signUpSource       447 non-null    object 
 3   state              439 non-null    object 
 4   _id_$oid           495 non-null    object 
 5   createdDate_$date  495 non-null    int64  
 6   lastLogin_$date    433 non-null    float64
dtypes: bool(1), float64(1), int64(1), object(4)
memory usage: 23.8+ KB


In [5]:
receipts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1119 entries, 0 to 1118
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bonusPointsEarned        544 non-null    float64
 1   bonusPointsEarnedReason  544 non-null    object 
 2   pointsEarned             609 non-null    object 
 3   purchasedItemCount       635 non-null    float64
 4   rewardsReceiptItemList   679 non-null    object 
 5   rewardsReceiptStatus     1119 non-null   object 
 6   totalSpent               684 non-null    object 
 7   userId                   1119 non-null   object 
 8   _id_$oid                 1119 non-null   object 
 9   createDate_$date         1119 non-null   int64  
 10  dateScanned_$date        1119 non-null   int64  
 11  finishedDate_$date       568 non-null    float64
 12  modifyDate_$date         1119 non-null   int64  
 13  pointsAwardedDate_$date  537 non-null    float64
 14  purchaseDate_$date      

#### ALl the 3 datasets contain a mixture of string(oject in Pandas) and numeric columns.

In [61]:
print('Receipts')
print('--------------------------')
print(receipts_df.isnull().sum())
print('--------------------------')
print('Users')
print('--------------------------')
print(users_df.isnull().sum())
print('--------------------------')
print('Brands')
print('--------------------------')
print(brands_df.isnull().sum())

Receipts
--------------------------
bonusPointsEarned          575
bonusPointsEarnedReason    575
pointsEarned               510
purchasedItemCount         484
rewardsReceiptItemList     440
rewardsReceiptStatus         0
totalSpent                 435
userId                       0
_id_$oid                     0
createDate_$date             0
dateScanned_$date            0
finishedDate_$date         551
modifyDate_$date             0
pointsAwardedDate_$date    582
purchaseDate_$date         448
dtype: int64
--------------------------
Users
--------------------------
active                0
role                  0
signUpSource         48
state                56
_id_$oid              0
createdDate_$date     0
lastLogin_$date      62
dtype: int64
--------------------------
Brands
--------------------------
barcode           0
category         54
categoryCode    229
name              0
topBrand        200
_id_$oid          0
cpg_$id_$oid      0
cpg_$ref          0
brandCode        83
dtyp

#### The Users Dataset has very few rows containing null values, but the dataset brands and receipts have plenty of rows having null values, If the columns containing null values are not relevant to the analysis, it is ok to leave them as is, but otherwise they must be filled with a substitue value or should be dropped if the dataset is big enough.

In [7]:
print(users_df.duplicated().sum()) # duplicates in users table
print(brands_df.duplicated().sum())
print(receipts_df.duplicated(subset = ['_id_$oid']).sum())

283
0
0


#### Out of 495 users in the dataset, 283 of them are duplicates.

In [191]:
#create a dict to store the itemlist against its receiptId
receipt_items = {}
for index , receipt in receipts_df.iterrows():
    #check for nan
    if receipt['rewardsReceiptItemList'] != receipt['rewardsReceiptItemList']:
        pass
    else:
        receipt_items[receipt['_id_$oid']] = receipt['rewardsReceiptItemList']

In [192]:
#create a pandas dataframe to flatten the receiptsRewardItemList and add a receiptId column with it
current_row = {}
itemlist_df = pd.DataFrame()

for key in receipt_items:
    for val in receipt_items[key]:
        current_row['receiptId'] = key
        for i in val:
            current_row[i] = val[i]
    itemlist_df = itemlist_df.append(current_row , ignore_index = True)
itemlist_df.head()

,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,receiptId,userFlaggedBarcode,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
0,4011,ITEM NOT FOUND,26.00,26.00,0.0,1,1.0,5.0,5ff1e1eb0a720f0523000575,4011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,1.0,2,1.0,1.0,5ff1e1bb0a720f052300056b,028400642255,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,0.0,1,1.0,1.0,5ff1e1f10a720f052300057a,4011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4011,ITEM NOT FOUND,28.00,28.00,0.0,1,1.0,4.0,5ff1e1ee0a7214ada100056f,4011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1234,ITEM NOT FOUND,2.56,2.56,1.0,2,1.0,3.0,5ff1e1d20a7214ada1000561,1234,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [193]:
itemlist_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 679 entries, 0 to 678
Data columns (total 35 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   barcode                             679 non-null    object 
 1   description                         679 non-null    object 
 2   finalPrice                          679 non-null    object 
 3   itemPrice                           679 non-null    object 
 4   needsFetchReview                    679 non-null    float64
 5   partnerItemId                       679 non-null    object 
 6   preventTargetGapPoints              679 non-null    float64
 7   quantityPurchased                   679 non-null    float64
 8   receiptId                           679 non-null    object 
 9   userFlaggedBarcode                  679 non-null    object 
 10  userFlaggedNewItem                  679 non-null    float64
 11  userFlaggedPrice                    679 non-n

In [194]:
# count of items scanned for each brandcode
itemlist_df['brandCode'].value_counts()

BRAND                        196
MISSION                      121
KRAFT EASY CHEESE             59
WINGSTOP                      47
BEN AND JERRYS                38
GERM-X                        33
AMERICAN BEAUTY               27
BORDEN                        24
KLEENEX                       24
CALIFIA FARMS                 15
KLARBRUNN                     14
KRUSTEAZ                      12
VIVA                           9
OSCAR MAYER                    7
HUGGIES                        7
GALLO FAMILY VINEYARDS         7
KIT KAT                        7
FOLGERS                        4
KNORR                          3
DORITOS                        2
MCCORMICK GRILL MATES          1
THOMAS ENGLISH MUFFINS         1
KLONDIKE                       1
THAI KITCHEN                   1
WONDERFUL                      1
STOUFFER'S CLASSICS            1
HY-VEE                         1
DOLE CHILLED FRUIT JUICES      1
HERSHEY'S KISSES               1
PEPSI                          1
HARVEST SN

In [195]:
#count of receipts for each receipt status
receipts_df['rewardsReceiptStatus'].value_counts() # no receipts for accepted status

FINISHED     518
SUBMITTED    434
REJECTED      71
PENDING       50
FLAGGED       46
Name: rewardsReceiptStatus, dtype: int64

### Create a SQLite database and create tables based on the data model

In [9]:
def connect_database(db):
    '''connect to the database mentioned in arguements'''
    conn = None
    try:
        conn = sqlite3.connect(db)
    except Error as e:
        print(e)
    return conn

In [10]:
def create_table(conn, create_table_sql):
    """create a table from the create_table_sql statement"""
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

In [11]:
def insert_records(conn, insert_sql, data):
    '''inserts data in tables mentioned in insert sql statement'''
    try:
        cur = conn.cursor()
        for row in data:
            cur.execute(insert_sql, row)
        conn.commit()
    except Error as e:
        print(e)
    return cur.lastrowid

In [50]:
def select_query(conn, select_sql):
    '''executes a select query'''
    try:
        cur = conn.cursor()
        cur.execute(select_sql)
        columns = [col[0] for col in cur.description]
        print(columns)
        rows = cur.fetchall()
        for row in rows:
            print(row)
    except Error as e:
        print(e)

In [13]:
#connect to the database
conn = connect_database(r'D:\Fetch Rewards - AE\rewards_db.db')

In [124]:
#create tables receipts and rewardpoints (can create rest of the tables similarly)

create_receipts_table_sql = '''CREATE TABLE Receipts(
                                receiptId Text PRIMARY KEY,
                                bonusPointsEarnedReason Text,
                                rewardsReceiptStatus Text                                
                                );'''

create_rewardpoints_table_sql = '''CREATE TABLE RewardsPoints(
                        receiptId Text,
                        userId Text,
                        createdDateId Number,
                        dateScannedId Number,
                        finishedDateId Number,
                        modifyDateId Number,
                        pointsAwardedDateId Number,
                        purchaseDateId Number,
                        bonusPointsEarned Number,
                        pointsEarned Number,
                        purchaseItemCount Number,
                        totalSpent Number,
                        FOREIGN KEY (receiptId) REFERENCES Receipts (receiptId)
                        );'''

if conn is not None:
    create_table(conn, create_receipts_table_sql)
    create_table(conn, create_rewardpoints_table_sql)
else:
    print('Error in connecting to the database!')


In [38]:
# insert data from pandas dataframe to database tables

insert_into_receipts = 'INSERT INTO Receipts(receiptId, bonusPointsEarnedReason, rewardsReceiptStatus) VALUES(?, ?, ?);'
insert_into_rewardpoints = '''INSERT INTO RewardsPoints(receiptId, userId, bonusPointsEarned, pointsEarned, purchaseItemCount, totalSpent)
                            VALUES(?, ?, ?, ?, ?, ?);'''

receipts_data = zip(receipts_df['_id_$oid'], receipts_df['bonusPointsEarnedReason'], receipts_df['rewardsReceiptStatus'])
rewardpoints_data = zip(receipts_df['_id_$oid'], receipts_df['userId'], receipts_df['bonusPointsEarned'], receipts_df['pointsEarned'], receipts_df['purchasedItemCount'], receipts_df['totalSpent'])

if conn is not None:
    insert_records(conn, insert_into_receipts, receipts_data)
    insert_records(conn, insert_into_rewardpoints, rewardpoints_data)
else:
    print('Cannot Insert Records. Error in connecting to the database!')

1119

In [197]:
# sql queries for predefined business questions

# average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

avg_spent = '''SELECT r.rewardsReceiptStatus, AVG(rp.totalSpent) AS avgTotalSpent
            FROM RewardsPoints AS rp JOIN Receipts AS r ON rp.receiptId = r.receiptId
            GROUP BY r.rewardsReceiptStatus
            HAVING r.rewardsReceiptStatus IN ('ACCEPTED', 'REJECTED', 'FINISHED')
            ORDER BY AVG(rp.totalSpent);'''

# total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

total_num_items = '''SELECT r.rewardsReceiptStatus, SUM(rp.purchaseItemCount) AS totalNumItems
                    FROM RewardsPoints AS rp JOIN Receipts AS r ON rp.receiptId = r.receiptId
                    GROUP BY r.rewardsReceiptStatus
                    HAVING r.rewardsReceiptStatus IN ('ACCEPTED', 'REJECTED', 'FINISHED')
                    ORDER BY SUM(rp.purchaseItemCount)
                '''

if conn is not None:
    select_query(conn, avg_spent)
    print('--------------------')
    select_query(conn, total_num_items)
else:
    print('Cannot execute the query. Error in connecting to the database!')


['rewardsReceiptStatus', 'avgTotalSpent']
('REJECTED', 23.326056338028184)
('FINISHED', 80.85430501930502)
--------------------
['rewardsReceiptStatus', 'totalNumItems']
('REJECTED', 173)
('FINISHED', 8184)


#### Since the dataset did not have any receipts for Accepted status, I included receipts of Finished status and it looks like receipts with status finished have the greater average spend as well as the total number of items purchased.